# Digital Twin Simulation - Full Pipeline Demo with Evaluation

This notebook demonstrates the complete digital twin simulation pipeline, including:
1. Downloading the dataset
2. Processing personas and questions
3. Running LLM simulations
4. Converting results to CSV format
5. Running accuracy evaluations
6. Performing pricing analysis

**Important**: 
- This notebook should be run from the `notebooks/` directory
- Make sure you have activated the poetry environment: `poetry shell`
- Ensure you have set your OPENAI_API_KEY in the `.env` file at the project root

**Note**: This notebook runs the full pipeline with a small subset of personas for demonstration. For production use, follow the shell scripts in the README.

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
import sys
import json
import yaml
from pathlib import Path
from dotenv import load_dotenv
import time

# Direct path setup - adjust this path if your project is in a different location
PROJECT_ROOT_PATH = "/Users/zg2467/gitdirs/Digital-Twin-Simulation"

# Set up project root
project_root = Path(PROJECT_ROOT_PATH)

# Verify the project root exists and has expected directories
if not project_root.exists():
    raise RuntimeError(f"Project root not found at: {project_root}")

if not (project_root / 'text_simulation').exists():
    raise RuntimeError(f"'text_simulation' directory not found in: {project_root}")

if not (project_root / 'evaluation').exists():
    raise RuntimeError(f"'evaluation' directory not found in: {project_root}")

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Configuration
MAX_PERSONAS = 50  # Limit for demo purposes

print(f"✅ Project root: {project_root}")
print(f"Current directory: {Path.cwd()}")
print(f"Python path configured: {sys.path[0]}")

# Set notebook directory
notebook_dir = project_root / 'notebooks'

In [ ]:
# Setup environment
print("=" * 60)
print("Digital Twin Simulation - Full Pipeline Demo")
print("=" * 60)
print()

# Load environment variables
load_dotenv(project_root / '.env')

# Check OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Please set your OPENAI_API_KEY in the .env file")
else:
    print("✅ OpenAI API key loaded successfully")

print(f"\nConfigured to process {MAX_PERSONAS} personas for this demo")

## 2. Download Dataset

First, we'll download the Twin-2K-500 dataset from Hugging Face.

In [ ]:
print("=" * 60)
print("Step 1: Download Dataset")
print("=" * 60)

data_dir = project_root / "data"
if (data_dir / "mega_persona_json" / "mega_persona").exists():
    print("✅ Dataset already downloaded")
else:
    print("Downloading dataset...")
    # Save current directory
    original_cwd = Path.cwd()
    
    try:
        # Change to project root for download
        os.chdir(project_root)
        
        # Import and run the download function directly
        import download_dataset
        download_dataset.main()
        print("✅ Dataset downloaded successfully")
    except Exception as e:
        print(f"❌ Error downloading dataset: {e}")
    finally:
        # Restore original directory
        os.chdir(original_cwd)

## 3. Update Configuration

Update the configuration file to limit processing to our demo size.

In [ ]:
print("=" * 60)
print("Step 2: Update Configuration")
print("=" * 60)

config_path = project_root / "text_simulation" / "configs" / "openai_config.yaml"

try:
    # Read current config
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    # Update max_personas
    config['max_personas'] = MAX_PERSONAS
    
    # Write back
    with open(config_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"✅ Updated config to process {MAX_PERSONAS} personas")
except Exception as e:
    print(f"❌ Error updating config: {e}")

## 4. Convert Personas to Text Format

Convert the JSON persona data into text format suitable for LLM processing.

In [ ]:
print("=" * 60)
print("Step 3: Convert Personas to Text")
print("=" * 60)

# Import the conversion function
from text_simulation.convert_persona_to_text import convert_persona_to_text
from tqdm import tqdm

# Set up paths
persona_json_dir = project_root / "data" / "mega_persona_json" / "mega_persona"
output_text_dir = project_root / "text_simulation" / "text_personas"

# Create output directory
output_text_dir.mkdir(parents=True, exist_ok=True)

try:
    # Get persona files and limit to MAX_PERSONAS for demo
    json_files = [f for f in os.listdir(persona_json_dir) 
                  if f.endswith('.json') and f.startswith('pid_')]
    
    # Limit files for demo
    files_to_process = json_files[:MAX_PERSONAS]
    
    print(f"Converting {len(files_to_process)} personas (limited for demo)...")
    
    successful = 0
    failed = 0
    
    for json_file in tqdm(files_to_process, desc="Converting personas"):
        input_path = persona_json_dir / json_file
        output_path = output_text_dir / json_file.replace('.json', '.txt')
        
        if convert_persona_to_text(str(input_path), str(output_path), "full"):
            successful += 1
        else:
            failed += 1
            print(f"Failed to convert {json_file}")
    
    print(f"\n✅ Conversion complete. Successful: {successful}, Failed: {failed}")
    
    # Check output directory
    persona_files = list(output_text_dir.glob("*.txt"))
    print(f"   Created {len(persona_files)} persona text files")
    
except Exception as e:
    print(f"❌ Error converting personas: {e}")

## 5. Convert Questions to Text Format

Process the survey questions into a format suitable for the simulation.

In [ ]:
print("=" * 60)
print("Step 4: Convert Questions to Text")
print("=" * 60)

# Use subprocess to run the script with proper Python path
import subprocess

# Run the script with PYTHONPATH set to include text_simulation directory
env = os.environ.copy()
env['PYTHONPATH'] = str(project_root / 'text_simulation') + os.pathsep + env.get('PYTHONPATH', '')

result = subprocess.run(
    [sys.executable, str(project_root / "text_simulation" / "convert_question_json_to_text.py")],
    cwd=str(project_root),
    env=env,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Questions converted successfully")
    
    # Check output
    output_dir = project_root / "text_simulation" / "text_questions"
    if output_dir.exists():
        question_files = list(output_dir.glob("*.txt"))
        print(f"   Created {len(question_files)} question text files")
else:
    print(f"❌ Error converting questions: {result.stderr}")
    # If it still fails, suggest manual fix
    print("\nNote: If this continues to fail, you can run manually:")
    print(f"  cd {project_root}")
    print("  python text_simulation/convert_question_json_to_text.py")

## 6. Create Simulation Input

Combine personas with questions to create the input files for simulation.

In [ ]:
print("=" * 60)
print("Step 5: Create Simulation Input")
print("=" * 60)

# Import the function
from text_simulation.create_text_simulation_input import create_combined_prompts

# Set up paths
persona_text_dir = str(project_root / "text_simulation" / "text_personas")
question_prompts_dir = str(project_root / "text_simulation" / "text_questions")
output_combined_prompts_dir = str(project_root / "text_simulation" / "text_simulation_input")

try:
    create_combined_prompts(
        persona_text_dir=persona_text_dir,
        question_prompts_dir=question_prompts_dir,
        output_combined_prompts_dir=output_combined_prompts_dir
    )
    
    print("✅ Simulation input created successfully")
    
    # Check how many input files were created
    input_dir = Path(output_combined_prompts_dir)
    if input_dir.exists():
        prompt_files = list(input_dir.glob("*_prompt.txt"))
        print(f"   Created {len(prompt_files)} prompt files")
        
        # Limit to MAX_PERSONAS for demo
        if len(prompt_files) > MAX_PERSONAS:
            print(f"   (Will process only first {MAX_PERSONAS} for this demo)")
    
except Exception as e:
    print(f"❌ Error creating simulation input: {e}")

## 7. Run LLM Simulations

Now we'll run the actual LLM simulations. This will use the OpenAI API to generate responses.

In [ ]:
# Display current configuration
config_path = project_root / "text_simulation" / "configs" / "openai_config.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Current simulation configuration:")
print(f"  Model: {config['model_name']}")
print(f"  Temperature: {config['temperature']}")
print(f"  Max personas: {config['max_personas']}")
print(f"  Workers: {config['num_workers']}")
print(f"  Force regenerate: {config['force_regenerate']}")

In [ ]:
print("=" * 60)
print("Step 6: Run LLM Simulations")
print("=" * 60)

print("\nRunning LLM simulations...")
print("This may take a few minutes depending on the number of personas and API rate limits.\n")

# Use subprocess to run the simulation with proper Python path
import subprocess

# Set up environment with proper PYTHONPATH
env = os.environ.copy()
env['PYTHONPATH'] = str(project_root) + os.pathsep + str(project_root / 'text_simulation') + os.pathsep + env.get('PYTHONPATH', '')

# Run the simulation script
process = subprocess.Popen(
    [
        sys.executable, 
        str(project_root / "text_simulation" / "run_LLM_simulations.py"),
        "--config", str(project_root / "text_simulation" / "configs" / "openai_config.yaml"),
        "--max_personas", str(MAX_PERSONAS)
    ],
    cwd=str(project_root),
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Stream output
try:
    for line in process.stdout:
        print(line.rstrip())
    
    process.wait()
    
    if process.returncode == 0:
        print("\n✅ Simulations completed successfully")
    else:
        print(f"\n❌ Error running simulations (exit code: {process.returncode})")
except KeyboardInterrupt:
    print("\n⚠️  Simulation interrupted by user")
    process.terminate()
    process.wait()

## 8. Examine Simulation Results

Let's look at some of the generated responses.

In [ ]:
print("=" * 60)
print("Step 7: Examine Results")
print("=" * 60)

output_dir = project_root / "text_simulation" / "text_simulation_output"

if output_dir.exists():
    persona_dirs = [d for d in output_dir.iterdir() if d.is_dir() and d.name.startswith("pid_")]
    print(f"Found {len(persona_dirs)} persona output directories\n")
    
    # Show a sample response
    if persona_dirs:
        sample_dir = persona_dirs[0]
        response_files = list(sample_dir.glob("*_response.json"))
        
        if response_files:
            with open(response_files[0], 'r') as f:
                response = json.load(f)
            
            print(f"Sample response from {sample_dir.name}:")
            print("=" * 50)
            print(f"Question ID: {response.get('question_id', 'N/A')}")
            print(f"\nPrompt (first 200 chars):")
            print(response.get('prompt_text', '')[:200] + "...")
            print(f"\nResponse (first 500 chars):")
            response_text = response.get('response_text', 'No response')
            if len(response_text) > 500:
                print(response_text[:500] + "...")
            else:
                print(response_text)
            print("=" * 50)
else:
    print("No output directory found")

## 9. Convert JSON to CSV for Evaluation

Convert the JSON answer blocks to CSV format for evaluation purposes.

In [ ]:
print("=" * 60)
print("Step 8: Convert JSON to CSV for Evaluation")
print("=" * 60)

# Create evaluation config for json2csv
eval_config = {
    "trial_dir": "text_simulation/text_simulation_output/",
    "model_name": "gpt-4.1-mini",
    "max_personas": MAX_PERSONAS,
    "waves": {
        "wave1_3": {
            "input_pattern": "data/mega_persona_json/answer_blocks/pid_{pid}_wave4_Q_wave1_3_A.json",
            "output_csv": "${trial_dir}/csv_comparison/responses_wave1_3.csv",
            "output_csv_formatted": "${trial_dir}/csv_comparison/csv_formatted/responses_wave1_3_formatted.csv",
            "output_csv_labeled": "${trial_dir}/csv_comparison/csv_formatted_label/responses_wave1_3_label_formatted.csv"
        },
        "wave4": {
            "input_pattern": "data/mega_persona_json/answer_blocks/pid_{pid}_wave4_Q_wave4_A.json",
            "output_csv": "${trial_dir}/csv_comparison/responses_wave4.csv",
            "output_csv_formatted": "${trial_dir}/csv_comparison/csv_formatted/responses_wave4_formatted.csv",
            "output_csv_labeled": "${trial_dir}/csv_comparison/csv_formatted_label/responses_wave4_label_formatted.csv"
        },
        "llm_imputed": {
            "input_pattern": "${trial_dir}/answer_blocks_llm_imputed/pid_{pid}_wave4_Q_wave4_A.json",
            "output_csv": "${trial_dir}/csv_comparison/responses_llm_imputed.csv",
            "output_csv_formatted": "${trial_dir}/csv_comparison/csv_formatted/responses_llm_imputed_formatted.csv",
            "output_csv_labeled": "${trial_dir}/csv_comparison/csv_formatted_label/responses_llm_imputed_label_formatted.csv"
        }
    },
    "benchmark_csv": "data/wave_csv/wave_4_numbers_anonymized.csv",
    "column_mapping": "evaluation/column_mapping.csv",
    "save_question_mapping": True,
    "question_mapping_output": "${trial_dir}/csv_comparison/question_mapping.csv",
    "generate_randdollar_breakdown": True,
    "randdollar_output": "${trial_dir}/csv_comparison/randdollar_breakdown.csv"
}

# Write temporary config file
temp_eval_config = project_root / "temp_eval_config.yaml"
with open(temp_eval_config, 'w') as f:
    yaml.dump(eval_config, f)

print("Converting JSON results to CSV format...")

# Run json2csv conversion
result = subprocess.run(
    [sys.executable, "evaluation/json2csv.py", "--config", str(temp_eval_config), "--all", "--verbose"],
    cwd=str(project_root),
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ JSON to CSV conversion completed successfully")
    
    # Check what was created
    csv_dir = project_root / "text_simulation" / "text_simulation_output" / "csv_comparison"
    if csv_dir.exists():
        csv_files = list((csv_dir / "csv_formatted").glob("*.csv")) if (csv_dir / "csv_formatted").exists() else []
        print(f"   Generated {len(csv_files)} formatted CSV files")
        if csv_files:
            print("   Files created:")
            for f in csv_files[:5]:  # Show first 5 files
                print(f"     - {f.name}")
else:
    print(f"⚠️  JSON to CSV conversion encountered issues")
    print(f"   Error: {result.stderr[:500]}...")  # Show first 500 chars of error
    
# Clean up temp config
if temp_eval_config.exists():
    temp_eval_config.unlink()

print(f"\nOutput directory: {csv_dir}")

## 10. MAD Accuracy Evaluation

Run Mean Absolute Difference (MAD) accuracy evaluation to compare simulated responses with ground truth.

In [ ]:
print("=" * 60)
print("Step 9: MAD Accuracy Evaluation")
print("=" * 60)

# Set up paths for MAD evaluation
trial_dir = project_root / "text_simulation" / "text_simulation_output"
csv_dir = trial_dir / "csv_comparison" / "csv_formatted"
output_dir = trial_dir / "accuracy_evaluation"
output_dir.mkdir(parents=True, exist_ok=True)

# Create MAD evaluation config
mad_config = {
    "csv_dir": str(csv_dir),
    "output_dir": str(output_dir),
    "output_excel_filename": "mad_accuracy_summary.xlsx",
    "output_plot_filename": "accuracy_dist.png",
    "plot_title": "Digital Twin Simulation - GPT-4.1-mini"
}

# Write temporary config file
temp_mad_config = project_root / "temp_mad_config.yaml"
with open(temp_mad_config, 'w') as f:
    yaml.dump(mad_config, f)

print("Computing MAD accuracy metrics...")

# Check if required CSV files exist
required_files = ["responses_wave1_3_formatted.csv", "responses_wave4_formatted.csv", "responses_llm_imputed_formatted.csv"]
missing_files = [f for f in required_files if not (csv_dir / f).exists()]

if missing_files:
    print(f"⚠️  Missing required CSV files: {missing_files}")
    print("   Skipping MAD evaluation...")
else:
    # Run MAD evaluation
    result = subprocess.run(
        [sys.executable, "evaluation/mad_accuracy_evaluation.py", "--config", str(temp_mad_config), "--verbose"],
        cwd=str(project_root),
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ MAD evaluation completed successfully")
        print(f"   Results saved to: {output_dir}")
        
        # Check outputs
        excel_file = output_dir / "mad_accuracy_summary.xlsx"
        plot_file = output_dir / "accuracy_dist.png"
        
        if excel_file.exists():
            print(f"   ✅ Excel summary: {excel_file.name}")
        if plot_file.exists():
            print(f"   ✅ Accuracy plot: {plot_file.name}")
            
            # Try to display the plot if in notebook environment
            try:
                from IPython.display import Image, display
                display(Image(str(plot_file)))
            except:
                print("   (Plot saved but cannot display inline)")
    else:
        print(f"⚠️  MAD evaluation encountered issues")
        print(f"   Error: {result.stderr[:500]}...")

# Clean up temp config
if temp_mad_config.exists():
    temp_mad_config.unlink()

## 11. Within-Between Subjects Analysis

This analysis examines behavioral economics experiments across different waves.

In [ ]:
print("=" * 60)
print("Step 10: Within-Between Subjects Analysis")
print("=" * 60)

# Import the analysis classes directly
from evaluation.within_between_subjects import (
    DataLoader, ExcelWriter, BaseRateAnalysis, OutcomeBiasAnalysis,
    FalseConsensusAnalysis, SunkCostAnalysis, AllaisProblemAnalysis,
    NonseparabilityAnalysis, FramingAnalysis, LindaProblemAnalysis,
    AnchoringAnalysis, RelativeSavingsAnalysis, MysideBiasAnalysis,
    OmissionBiasAnalysis, LessIsMoreAnalysis, ThalerProblemAnalysis,
    ProbabilityMatchingAnalysis, DenominatorNeglectAnalysis
)

print("Running within-between subjects analysis...")
print("This analysis examines behavioral economics experiments across waves.")

# Save current directory and change to project root for the analysis
original_cwd = Path.cwd()
os.chdir(project_root)

try:
    # Set up data loader (now with correct relative paths from project root)
    trial_dir_str = str(trial_dir.relative_to(project_root))
    data_loader = DataLoader(trial_dir_str)
    
    # Get common IDs (respondents who completed all 4 waves)
    try:
        common_ids = data_loader.get_common_ids()
        print(f"\n✅ Found {len(common_ids)} respondents who completed all 4 waves")
    except Exception as e:
        print(f"⚠️  Error finding common respondents: {e}")
        print("   This analysis requires the full dataset with all waves")
        common_ids = set()
    
    if len(common_ids) > 0:
        # Load data for each wave
        print("\nLoading wave data...")
        try:
            data = {
                "wave1": data_loader.load_wave_data("wave1", common_ids),
                "wave2": data_loader.load_wave_data("wave2", common_ids),
                "wave3": data_loader.load_wave_data("wave3", common_ids),
                "wave4": data_loader.load_wave_data("wave4", common_ids),
                "LLM": data_loader.load_wave_data("LLM", common_ids)
            }
            
            for wave, df in data.items():
                if df is not None and not df.empty:
                    print(f"   {wave}: {len(df)} responses loaded")
                else:
                    print(f"   {wave}: No data loaded")
            
        except Exception as e:
            print(f"⚠️  Error loading wave data: {e}")
            data = {}
        
        # List of all analyses
        analyses_to_run = [
            ("Base Rate Neglect", BaseRateAnalysis),
            ("Outcome Bias", OutcomeBiasAnalysis),
            ("False Consensus Effect", FalseConsensusAnalysis),
            ("Sunk Cost Fallacy", SunkCostAnalysis),
            ("Allais Paradox", AllaisProblemAnalysis),
            ("Non-separability of Risks", NonseparabilityAnalysis),
            ("Framing Effects", FramingAnalysis),
            ("Linda Problem (Conjunction Fallacy)", LindaProblemAnalysis),
            ("Anchoring and Adjustment", AnchoringAnalysis),
            ("Relative vs Absolute Savings", RelativeSavingsAnalysis),
            ("Myside Bias", MysideBiasAnalysis),
            ("Omission Bias", OmissionBiasAnalysis),
            ("Less is More Effect", LessIsMoreAnalysis),
            ("Thaler Problem (WTA/WTP)", ThalerProblemAnalysis),
            ("Probability Matching vs Maximizing", ProbabilityMatchingAnalysis),
            ("Denominator Neglect", DenominatorNeglectAnalysis)
        ]
        
        # Run a sample of analyses and display results
        print(f"\nRunning {len(analyses_to_run)} behavioral economics analyses...")
        print("\nSample Results:")
        print("-" * 60)
        
        # For demo, run just a few analyses and show their results
        sample_analyses = analyses_to_run[:3]  # Run first 3 analyses for demo
        
        for analysis_name, analysis_class in sample_analyses:
            print(f"\n📊 {analysis_name}:")
            try:
                # Create a mock Excel writer that captures results
                class MockExcelWriter:
                    def __init__(self):
                        self.results = []
                    
                    def get_unique_sheet_name(self, base_name):
                        return base_name
                    
                    def write_results(self, sheet_name, results, header_note=None):
                        self.results = results
                        if header_note:
                            # Fix: Extract first line without using backslash in f-string
                            first_line = header_note.split('\n')[0][:100]
                            print(f"   Description: {first_line}...")
                
                mock_writer = MockExcelWriter()
                analysis = analysis_class(mock_writer)
                
                # Run the analysis
                analysis.run(data)
                
                # Display results
                if mock_writer.results:
                    for title, df in mock_writer.results[:2]:  # Show first 2 results per analysis
                        if title and not df.empty:
                            print(f"\n   {title}:")
                            print(df.to_string(index=False, max_rows=5))
                            if len(df) > 5:
                                print(f"   ... ({len(df)-5} more rows)")
                else:
                    print("   No results generated")
                    
            except Exception as e:
                print(f"   ⚠️  Error: {str(e)[:100]}")
                if "specific experimental questions" in str(e) or "KeyError" in str(e):
                    print("   This analysis requires specific questions that may not be in the limited demo")
        
        print("\n" + "-" * 60)
        print(f"\n✅ Completed sample analyses. In the full pipeline, all {len(analyses_to_run)} analyses")
        print("   would be run and saved to an Excel file with detailed results.")
        
        # Show where full results would be saved
        excel_output_dir = project_root / "text_simulation" / "text_simulation_output" / "accuracy_evaluation"
        output_filename = excel_output_dir / "within_subject_analysis.xlsx"
        print(f"\n   Full results would be saved to: {output_filename}")
        
    else:
        print("\n⚠️  No common respondents found across all waves")
        print("   This analysis requires the full dataset with respondents who completed all 4 waves")
        print("   For the demo with limited personas, this analysis cannot be performed")
    
finally:
    # Always restore the original working directory
    os.chdir(original_cwd)

print("\nNote: Within-between subjects analysis examines various cognitive biases and")
print("behavioral economics phenomena across human responses and LLM simulations.")

## 12. Pricing Analysis

Analyze pricing decisions to generate demand curves.

In [ ]:
print("=" * 60)
print("Step 11: Pricing Analysis")
print("=" * 60)

# Import pricing analysis functions directly
from evaluation.pricing_analysis import (
    load_randdollar_breakdown,
    prepare_purchase_data,
    calculate_relative_prices
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set up paths for pricing analysis
label_dir = trial_dir / "csv_comparison" / "csv_formatted_label"
randdollar_file = trial_dir / "csv_comparison" / "randdollar_breakdown.csv"
output_plot = trial_dir / "pricing_analysis_results" / "average_demand_curve.png"
output_plot.parent.mkdir(parents=True, exist_ok=True)

print("Running pricing demand curve analysis...")

# Check if randdollar file exists
if not randdollar_file.exists():
    print(f"⚠️  Missing required file: {randdollar_file.name}")
    print("   This file is generated during JSON to CSV conversion")
    print("   Skipping pricing analysis...")
else:
    try:
        # Load the randdollar breakdown data
        print("Loading randdollar breakdown data...")
        df_randdollar_breakdown = load_randdollar_breakdown(str(randdollar_file))
        
        if df_randdollar_breakdown is None or df_randdollar_breakdown.empty:
            print("⚠️  Could not load randdollar breakdown data")
        else:
            print(f"   Loaded {len(df_randdollar_breakdown)} price observations")
            
            # Prepare purchase data for each wave
            print("Preparing purchase data for each wave...")
            data_wave3 = prepare_purchase_data(df_randdollar_breakdown, "Wave1-3")
            data_wave4 = prepare_purchase_data(df_randdollar_breakdown, "Wave4")
            data_llm = prepare_purchase_data(df_randdollar_breakdown, "LLM_Imputed")
            
            # Combine all purchase data
            all_purchase_data = pd.concat([data_wave3, data_wave4, data_llm], ignore_index=True)
            
            if all_purchase_data.empty:
                print("⚠️  No purchase data could be processed")
            else:
                print(f"   Prepared {len(all_purchase_data)} purchase observations")
                
                # Calculate relative prices
                print("Calculating relative prices...")
                all_purchase_data, nprices = calculate_relative_prices(all_purchase_data)
                
                if nprices == 0:
                    print("⚠️  Could not determine price ranks - cannot generate demand curves")
                else:
                    print(f"   Found {nprices} distinct price points")
                    
                    # Compute demand curves
                    print("Computing demand curves...")
                    demand_curves = {}
                    for wave_name in ["Wave1-3", "Wave4", "LLM_Imputed"]:
                        current_wave_data = all_purchase_data[all_purchase_data["Wave"] == wave_name]
                        if not current_wave_data.empty:
                            curve = current_wave_data.groupby("Relative_Price_Rank")["Purchase"].mean()
                            # Reindex to ensure all price ranks from 1 to nprices are present
                            demand_curves[wave_name] = curve.reindex(range(1, nprices + 1), fill_value=np.nan)
                        else:
                            demand_curves[wave_name] = pd.Series([np.nan] * nprices, index=range(1, nprices + 1))
                    
                    # Create the plot
                    print("Creating demand curve plot...")
                    plt.figure(figsize=(10, 6))
                    x_axis = np.arange(1, nprices + 1)
                    
                    # Plot each wave's demand curve
                    if not demand_curves["Wave1-3"].isna().all():
                        plt.plot(x_axis, demand_curves["Wave1-3"], linestyle='-', marker='o', label='Wave 1-3')
                    if not demand_curves["Wave4"].isna().all():
                        plt.plot(x_axis, demand_curves["Wave4"], linestyle=':', marker='s', label='Wave 4')
                    if not demand_curves["LLM_Imputed"].isna().all():
                        plt.plot(x_axis, demand_curves["LLM_Imputed"], linestyle='-.', marker='^', label='LLM Imputed (Twins)')
                    
                    plt.ylim(0, 1)
                    plt.xticks(x_axis, fontsize=12)
                    plt.yticks(fontsize=12)
                    plt.xlabel('Relative Price Rank', fontsize=16)
                    plt.ylabel('Purchase Probability', fontsize=16)
                    plt.title('Average Demand Curve by Relative Price', fontsize=20)
                    plt.legend(fontsize=12, loc='best')
                    plt.grid(True, linestyle='--', alpha=0.7)
                    
                    # Save the plot
                    plt.savefig(str(output_plot), dpi=300, bbox_inches='tight')
                    plt.close()
                    
                    print(f"✅ Pricing analysis completed successfully")
                    print(f"   Demand curve plot saved to: {output_plot.name}")
                    
                    # Try to display the plot in the notebook
                    try:
                        from IPython.display import Image, display
                        display(Image(str(output_plot)))
                    except:
                        print("   (Plot saved but cannot display inline)")
                        
    except Exception as e:
        print(f"⚠️  Pricing analysis encountered an error: {e}")
        print("   This may be due to insufficient pricing data in the limited demo")
        print("   For full analysis, use the complete dataset with all personas")

## Summary

This notebook demonstrated the complete digital twin simulation pipeline with evaluation.

In [ ]:
print("=" * 60)
print("Pipeline Complete!")
print("=" * 60)
print("\n📊 Summary of Results:\n")

# Simulation summary
print("1. SIMULATION:")
print(f"   ✅ Processed {MAX_PERSONAS} personas (demo limit)")
print(f"   ✅ Generated responses for survey questions")

# Check simulation outputs
sim_output_dir = project_root / "text_simulation" / "text_simulation_output"
if sim_output_dir.exists():
    persona_dirs = len([d for d in sim_output_dir.iterdir() if d.is_dir() and d.name.startswith("pid_")])
    print(f"   ✅ Created {persona_dirs} persona output directories")

# Evaluation summary
print("\n2. EVALUATION:")

# CSV conversion
csv_dir = project_root / "text_simulation" / "text_simulation_output" / "csv_comparison"
if (csv_dir / "csv_formatted").exists():
    csv_count = len(list((csv_dir / "csv_formatted").glob("*.csv")))
    print(f"   ✅ Generated {csv_count} CSV files for analysis")

# MAD accuracy
accuracy_dir = project_root / "text_simulation" / "text_simulation_output" / "accuracy_evaluation"
if (accuracy_dir / "mad_accuracy_summary.xlsx").exists():
    print("   ✅ MAD accuracy evaluation completed")
    print("      - Excel summary: mad_accuracy_summary.xlsx")
    print("      - Accuracy plot: accuracy_dist.png")

# Within-between analysis
if (accuracy_dir / "within_subject_analysis.xlsx").exists():
    print("   ✅ Within-between subjects analysis completed")

# Pricing analysis
pricing_dir = project_root / "text_simulation" / "text_simulation_output" / "pricing_analysis_results"
if pricing_dir.exists() and any(pricing_dir.glob("*.png")):
    print("   ✅ Pricing demand curve analysis completed")

print("\n3. KEY OUTPUTS:")
print(f"   📁 All results saved to: {sim_output_dir}")
print("   📊 Key directories:")
print(f"      - Simulation outputs: text_simulation_output/pid_*/")
print(f"      - CSV comparisons: text_simulation_output/csv_comparison/")
print(f"      - Accuracy metrics: text_simulation_output/accuracy_evaluation/")
print(f"      - Pricing analysis: text_simulation_output/pricing_analysis_results/")

print("\n4. NEXT STEPS:")
print("   • To process all 2058 personas, remove the MAX_PERSONAS limit")
print("   • For production runs, use the shell scripts:")
print("     - ./scripts/run_pipeline.sh (simulation)")
print("     - ./scripts/run_evaluation_pipeline.sh (evaluation)")
print("   • Review the evaluation metrics to assess digital twin quality")

print("\n" + "=" * 60)
print("✅ Digital Twin Simulation Pipeline Complete!")
print("=" * 60)